In [23]:
import pandas as pd 

df = pd.read_excel(r"C:\Users\kfq6\Documents\Data\Sammedag_master_HbA1c_Features.xlsx")
#drop rows where target variable is missing
df_model = df.dropna(subset=["HbA1c_next"]).copy()


In [24]:




# 2) Encode sex
df_model["sex_male"] = (df_model["sex"] == "M").astype(int)

# 3) One-hot diagnosis
diag_dummies = pd.get_dummies(df_model["diagnosis"], prefix="dx")
df_model = pd.concat([df_model, diag_dummies], axis=1)



In [25]:
# 4) Optional missingness indicators
for col in ["LABmean__u_albumin_kreatinin_ratio_mg_g",
            "HbA1c_slope_prev_year", "HbA1c_CV_prev_year", "HbA1c_MAC_prev_year"]:
    df_model[f"has_{col}"] = df_model[col].notna().astype(int)





In [26]:

# 5) Define feature columns
drop_cols = [
    "DW_EK_Borger",
    "sex",
    "diagnosis",
    "anchor_date",
    "HbA1c_next",
    "year_next",
    "anchor_date_next",
    "delta_years_to_next",
]

feature_cols = [c for c in df_model.columns if c not in drop_cols]

X = df_model[feature_cols]
y = df_model["HbA1c_next"]

In [27]:
from sklearn.model_selection import train_test_split

ID_COL = "DW_EK_Borger"
unique_ids = df_model[ID_COL].unique()

train_ids, test_ids = train_test_split(
    unique_ids,
    test_size=0.3,     # fx 20% test
    random_state=42
)

train_df = df_model[df_model[ID_COL].isin(train_ids)].copy()
test_df  = df_model[df_model[ID_COL].isin(test_ids)].copy()


In [37]:
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor


target_col = "HbA1c_next"

X_train = train_df[feature_cols]
y_train = train_df[target_col]
groups  = train_df[ID_COL]

gkf = GroupKFold(n_splits=5)

model = RandomForestRegressor(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)

cv_scores = cross_val_score(
    model,
    X_train,
    y_train,
    cv=gkf,
    scoring="neg_mean_absolute_error",
    groups=groups
)

print("CV MAE (per fold):", -cv_scores)
print("CV MAE (mean):", -cv_scores.mean())


CV MAE (per fold): [5.85104261 5.45364815 6.46075529 6.12729416 6.71551982]
CV MAE (mean): 6.12165200782375


In [38]:
model.fit(X_train, y_train)

X_test = test_df[feature_cols]
y_test = test_df[target_col]

from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

y_test_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_test_pred)
rmse = mean_squared_error(y_test, y_test_pred, squared=False)

print(f"TEST MAE:  {mae:.2f}")
print(f"TEST RMSE: {rmse:.2f}")


TEST MAE:  5.89
TEST RMSE: 8.05


c:\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [30]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_test_pred)
print(f"TEST R^2: {r2:.3f}")



TEST R^2: 0.459


In [31]:
y_true = np.array(y_test)
y_pred = np.array(y_test_pred)

ss_res = np.sum((y_true - y_pred) ** 2)        # residual sum of squares
ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)  # total sum of squares

r2_manual = 1 - ss_res / ss_tot
print("R^2 (manual):", r2_manual)


R^2 (manual): 0.4591254195204342


In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

ID_COL = "DW_EK_Borger"

# Groups = patients
groups = train_df[ID_COL]

gkf = GroupKFold(n_splits=5)

rf = RandomForestRegressor(
    random_state=42,
    n_jobs=-1
)

# Reasonable hyperparameter search space
param_dist = {
    "n_estimators":     [200, 300, 400, 600, 800],
    "max_depth":        [None, 5, 10, 15, 20],
    "min_samples_split":[2, 5, 10, 20],
    "min_samples_leaf": [1, 2, 4, 8],
    "max_features":     ["sqrt", "log2", 0.3, 0.5, 0.7],
    "bootstrap":        [True, False],
}

search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=80,              # bump to 80 if you’re feeling masochistic
    scoring="r2",           # or "neg_mean_absolute_error" if you want to optimise MAE
    cv=gkf,
    n_jobs=-1,
    random_state=42,
    verbose=2
)

search.fit(X_train, y_train, groups=groups)

print("Best params:", search.best_params_)
print("Best CV R^2:", search.best_score_)


Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best params: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 8, 'max_features': 0.7, 'max_depth': 10, 'bootstrap': True}
Best CV R^2: 0.5272990331896266


In [33]:
from xgboost import XGBRegressor

xgb = XGBRegressor(
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1,
    tree_method="hist",  # faster, if available
)

param_dist_xgb = {
    "n_estimators": [200, 400, 600, 800],
    "max_depth":    [3, 4, 5, 6],
    "learning_rate":[0.01, 0.03, 0.05, 0.1],
    "subsample":    [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "gamma":        [0, 0.1, 0.3],
    "min_child_weight": [1, 3, 5],
}

search_xgb = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist_xgb,
    n_iter=40,
    scoring="r2",
    cv=gkf,
    n_jobs=-1,
    random_state=42,
    verbose=2
)

search_xgb.fit(X_train, y_train, groups=groups)

print("Best XGB params:", search_xgb.best_params_)
print("Best XGB CV R^2:", search_xgb.best_score_)

best_xgb = search_xgb.best_estimator_

y_test_pred_xgb = best_xgb.predict(X_test)

r2_test_xgb  = r2_score(y_test, y_test_pred_xgb)
mae_test_xgb = mean_absolute_error(y_test, y_test_pred_xgb)
rmse_test_xgb = mean_squared_error(y_test, y_test_pred_xgb, squared=False)

print(f"XGB TEST R²:  {r2_test_xgb:.3f}")
print(f"XGB TEST MAE: {mae_test_xgb:.2f}")
print(f"XGB TEST RMSE:{rmse_test_xgb:.2f}")


Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best XGB params: {'subsample': 0.6, 'n_estimators': 400, 'min_child_weight': 3, 'max_depth': 5, 'learning_rate': 0.01, 'gamma': 0.1, 'colsample_bytree': 0.8}
Best XGB CV R^2: 0.5273896701232175
XGB TEST R²:  0.483
XGB TEST MAE: 5.76
XGB TEST RMSE:7.87


c:\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [34]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Global mean baseline
mean_baseline = y_train.mean()
y_test_pred_mean = np.full_like(y_test, fill_value=mean_baseline, dtype=float)

mae_mean = mean_absolute_error(y_test, y_test_pred_mean)
rmse_mean = mean_squared_error(y_test, y_test_pred_mean, squared=False)

print(f"Baseline (global mean) - MAE: {mae_mean:.2f}, RMSE: {rmse_mean:.2f}")

# "Predict current HbA1c" baseline
current_col = "LABmean__hb_b_haemoglobin_a1c_ifcc_mmol_mol"
y_test_pred_same = test_df[current_col].values

mae_same = mean_absolute_error(y_test, y_test_pred_same)
rmse_same = mean_squared_error(y_test, y_test_pred_same, squared=False)

print(f"Baseline (same as current) - MAE: {mae_same:.2f}, RMSE: {rmse_same:.2f}")


Baseline (global mean) - MAE: 8.71, RMSE: 11.06
Baseline (same as current) - MAE: 6.07, RMSE: 8.72


c:\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [35]:
import pandas as pd
import numpy as np

importances = best_xgb.feature_importances_
fi = pd.DataFrame({
    "feature": feature_cols,
    "importance": importances
}).sort_values("importance", ascending=False)

print(fi.head(20))


                                        feature  importance
12  LABmean__hb_b_haemoglobin_a1c_ifcc_mmol_mol    0.198026
0                                           age    0.038825
21      LABmean__u_albumin_kreatinin_ratio_mg_g    0.033392
29                           dx_Type 1-diabetes    0.032512
30                           dx_Type 2-diabetes    0.032326
11          LABmean__egfr_1_73m2_ckd_epi_ml_min    0.030595
19                LABmean__p_triglycerid_mmol_l    0.030088
25                          HbA1c_MAC_prev_year    0.030035
32                    has_HbA1c_slope_prev_year    0.029116
23                        HbA1c_slope_prev_year    0.028773
24                           HbA1c_CV_prev_year    0.028691
14             LABmean__p_kolesterol_hdl_mmol_l    0.028140
5                                     comp_foot    0.027555
1                                      comp_eye    0.026592
17                  LABmean__p_kreatinin_umol_l    0.026049
7                                      c